In [59]:
import os
from collections import OrderedDict

import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

% matplotlib inline

### WB Data Analysis

**<span style="color:red; background:yellow;"></span>**

**Data & Scope:** downloaded on (date/time)
Two types of datasets:

1) Population (easy)
* years 1960-2016
* definition: total # residents (Regardless of citizenship/legal status)
* sources: (mid-year value)

    1) United Nations Population Division. World Population Prospects
    
    2) Census reports and other statistical publications from national statistical offices
    
    3) Eurostat: Demographic Statistics
    
    4) United Nations Statistical Division. Population and Vita

2) GDP (slightly more tricky)
- years: 1960 - 2016
- choosing to look at `gdp` (market, constant) first.
- Other types that one can look at:
    * MKT current
    * PPP current
    * PPP constant

**Source:** World Bank

**Assumptions/Expectations**

**Analysis Goals**
- Task 3 Deliverable: ADM0 population & real income estimates from `1950-2017`  **<span style="color:gray; background:lime;">DONE</span>**

- Find out which countries are missing data for which years. 
    Population **<span style="color:gray; background:lime;">DONE</span>**
    GDP
- Or alternatively, which years are missing certain *countries*

Possible convenience functions:
* Find years missing for a given country (to check important countries)
* Create filter for population > 10 mil: look at countries exit/enter data on which years. (Mike's suggesiton)
* From metadata, get a list of country code that are not *countries* to filter them from the list **<span style="color:gray; background:lime;">DONE</span>**


**Conclusion**

#1. Population. Population data exists from **1960 to 2016.**
    
```  
Countries out of 217 that are missing population data (nan):

West Bank and Gaza: 1960 - 1989
Serbia: 1960 - 1989
Sint Maarten (Dutch part): 1960 -1998
Kuwait: 1992-1994
Eritrea: 2012-2016

```

#2. GDP 

```
Countries out of 217 that are missing gdp data

a. GPP market constant
b. GDO PPP constant

```



# Z. Convenience functions for both Population & Income data


Both population and income dataset includes list of countries (217) plus various other groups^^ (46 - regional, income or political) as entry data. 217 is the expected number of countries listed in World Bank, 2017.

Convenience functions:

1) From the original WB dataset, select only columns of interest

2) Get only country-specific GDP and population data (filter out other groups^^, mentioned above, data)

### 1) Select interested columns from data

In [60]:
def select_relevant_cols(_df):
    # returns df with 'Country Name', 'Country Code' and all years (1960 to 2016)
    _cols = ['Country Name'] + ['Country Code'] + [str(yr) for yr in range(1960, 2017)]
    return _df[_cols]

### 2) Filter out non_countries (i.e. get only country data)

In [61]:
def filter_non_countries(_df, _metadata):
    '''
    _df : pd.DataFrame
        either income or population data
        
    _metadata : pd.DataFrame
        metadata on a list of entries including countries and non-countries 
        data source is from the World Bank
        has IncomeGroup column that is not null for countries (217)
    '''
    # Country Code in both dataframes
    _merged = _df.merge(_metadata, on='Country Code')
    
    # non-countries have no IncomeGroup
    non_country_mask = _merged['IncomeGroup'].isnull()
    merged_country_only = _merged[~non_country_mask]
    return merged_country_only

# A. Population data

Total population (absolute units) - based on national census and extrapolation and interpolation for missing values (based on data from United Nations, other census organizations, Eurostats and WB methodology). Subject to undercounting/biases for both high and low/mid income countries. 

Interpolation and extrapolation done by World Bank/UN (??-confirm the responsible party) for certain years/countries that are missing census data, or missing pre/post census information for given time frame. Uses demographic models, etc. 

### Initial survey of data

#### Download population data + metadata

In [62]:
zerg_path = os.environ['ZERG']

In [63]:
df2 = pd.read_csv(zerg_path + 'data/wb/population/API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=4)
meta_country = pd.read_csv(zerg_path + 'data/wb/population/Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2.csv')

#del df2['Unnamed: 62'] # remove extraneous data
df2.columns
df2.tail(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17396367.0,17850045.0,18322335.0,18809939.0,19308166.0,19813947.0,...,5.025581e+07,5.097943e+07,5.172935e+07,5.250652e+07,5.331196e+07,5.414673e+07,5.501198e+07,55908865.0,NaN,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3044846.0,3140264.0,3240587.0,3345145.0,3452942.0,3563407.0,...,1.345642e+07,1.385003e+07,1.426476e+07,1.469994e+07,1.515321e+07,1.562097e+07,1.610059e+07,16591390.0,NaN,NaN
263,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,3747369.0,3870756.0,3999419.0,4132756.0,4269863.0,4410212.0,...,1.381060e+07,1.408632e+07,1.438665e+07,1.471083e+07,1.505451e+07,1.541168e+07,1.577745e+07,16150362.0,NaN,NaN



**WARNING**: had to re-download the file because I modified it by
opening and interacting with in via Excel. Produced error in read_csv call.

**TIP for myself**: Only Open a copy version in Excel


In [64]:
del meta_country['Unnamed: 5'] 
meta_country.columns

Index(['Country Code', 'Region', 'IncomeGroup', 'SpecialNotes', 'TableName'], dtype='object')

In [65]:
df2
df2['2017'].unique() # nan -> all 2017 values nan

array([ nan])

### Initial survey of metadata

In [66]:
meta_country['Region'].unique() # 7 excluding nan

'''['Latin America & Caribbean' 
 'South Asia' 
 'Sub-Saharan Africa'
 'Europe & Central Asia' /// nan 
 'Middle East & North Africa'
 'East Asia & Pacific' 
 'North America']

'''

#print(meta_country['IncomeGroup'].nunique()) #4
meta_country['IncomeGroup'].unique() 
'''['High income', 'Low income', 'Lower middle income', 'Upper middle income', nan]'''

meta_country['Country Code'].nunique() # 263 entries 
meta_country['IncomeGroup'].isnull().value_counts() # 217 False (countries) # 46 nan
# False values are countries -- i.e. all countries belong to an IncomeGroup
# Validation: we know from (World Bank 2017) that total of 217 countries were included

False    217
True      46
Name: IncomeGroup, dtype: int64

### Select relevant columns

In [67]:
df2 # includes Country Name and all years (1960-2016)
df2.shape # 264 entries, 58 columns 

(264, 63)

In [68]:
df2.head(5)   # Show top 5 rows

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,NaN,NaN


In [69]:
df2_countries_only = filter_non_countries(df2, meta_country)


In [70]:
df_pop = select_relevant_cols(df2_countries_only)
df_pop.shape

(217, 59)

### Find countries/years missing population data

In [71]:
df_pop.isnull().sum()
# note that nan values per year changes from 1-4, 1989 and prior are all 4,
# after that there is a shifts six times:
# 4 (60-89) -> 2 (90-91)-> 3 (92-94) -> 2 (95-97) -> 1 (98-2011)-> 2 (2012-2016)-> 264 (2017)

# Compare two years (diff countries)

df_pop['1989'].isnull()
list_of_indexes_with_nans = pd.isnull(df_pop).any(1).nonzero()[0]
list_of_indexes_with_nans

array([ 58, 106, 161, 176, 184])

NOTE on nans:

* `df.isnull().sum()`: returns total number of nan values for each column (whole df) <- what I used
* comparison of nan == np.nan to find nan values does not work (returns False)
* can use np.isnan(val) for a single value OR possibly use `apply` method for whole col (did not try)

In [72]:
def print_country_name_and_index(_df, _list_of_indexes_with_nans):
    _countries = [(_df.iloc[i]['Country Name'], i) for i in _list_of_indexes_with_nans]
    print (_countries)

In [73]:
def get_missing_years_by_idx(_df, idx):
    _country = _df.iloc[idx]
    _nan_years = _country.loc[_country.isnull()]
    print(_country['Country Name'], _nan_years.keys())

for _i in list_of_indexes_with_nans:
    get_missing_years_by_idx(df_pop, _i)

Eritrea Index(['2012', '2013', '2014', '2015', '2016'], dtype='object')
Kuwait Index(['1992', '1993', '1994'], dtype='object')
West Bank and Gaza Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'],
      dtype='object')
Serbia Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'],
      dtype='object')
Sint Maarten (Dutch part) Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
  

### Manual validation

In [74]:
#df_pop.iloc[58] # Eritrea missing 2012-2016
#df_pop.iloc[106] # Kuwait missing 1992-1994
#df_pop.iloc[161] # West Bank and Gaza missing 1960 - 1989
#df_pop.iloc[176] # Serbia missing 1960 - 1989
# df_pop.iloc[184] # Sint Maarten (Dutch part) 1960 -1997

In [75]:
#df_pop['1990'].isnull().sum() == 1 # expect 1 (excluding West Bank/Serbia)
#df_pop['1993'].isnull().sum() == 2 # expect 2 (Kuwait missing too)
#df_pop['1995'].isnull().sum() == 1 # expect 1 (Kuwait back again)
#df_pop['1999'].isnull().sum() # expect 0 Sint Maarten back
#df_pop['2012'].isnull().sum() # expect 1 (Eritrea goes missing)
#df_pop['2016'].isnull().sum() # expect 1

In [76]:
# doing this manual painful way for now

((df_pop['1960'].isnull().sum() == 3) 
 and (df_pop['1990'].isnull().sum() == 1) 
 and (df_pop['1993'].isnull().sum() == 2) 
 and (df_pop['1995'].isnull().sum() == 1)
 and df_pop['1999'].isnull().sum()==0
 and df_pop['2012'].isnull().sum()==1
 and df_pop['2016'].isnull().sum()==1)

True

##  Playing around with data

### Getting data for country (not really used)

In [77]:
df = df_pop

# Get row by country name
# country_row = _df.loc[_df['Country Name'] == country_name]

# Not yet working!
def get_missing_years(_df, country_name):
    '''
    Calculate the number of years the country has no
    data points (ie. has nan value)
    
    Parameters
    ----------
    country_name : Str
        name of given country
        
    Returns 
    -------
    Missing years in default range (1960-2017), where 
    each year is a string : List
    
    ex. ['1990',  '2005'']

'''
    # print list of years with missing data
    # print (_df.isnull().any().tolist())

    years_missing_data = _df.columns[_df.isnull().any()]
    return _df.loc[:, years_missing_data]
    #return country_row[country_row.isnull().any()].tolist()
    #return country

In [78]:
# Get countries as dataframe (subset of original) by first letter
def get_by_first_letter(df, first_letter):
    return df.loc[df['Country Name'].str.startswith(first_letter)]

# Get country by name
def get_row_by_country_name(_df, country_name):
    return _df[_df['Country Name'] == country_name]

# Get country row from dataframe by index
def get_by_index(df, country_idx):
    return df.iloc[country_idx]

# Get all countries that start with S
s_countries = get_by_first_letter(df, 'S')
#s_countries
    
# Get_by_first_letter(df, 'K') # example South Korea is index 124
# get_by_index(df, 124)


In [79]:
# Returns all values of the 'Country Name' column as an array
#df2['Country Name'].values #lists all countries
# United States
# China
# India

china = get_row_by_country_name(df2, 'China')
india = get_row_by_country_name(df2, 'India')
usa = get_row_by_country_name(df2, 'United States')
def get_first_and_last_years(_country):
    # _country is a dataframe (1 row)
    return _country[['1960', '2016']]
    
get_first_and_last_years(china)
get_first_and_last_years(usa)
get_first_and_last_years(india)

# manual confirmation: these values match online WB values (https://data.worldbank.org/indicator/SP.POP.TOTL?year_high_desc=true)

,1960,2016
107,449480608.0,1.324171e+09


# C. GDP data

### 1) PICK ONE: GDP (constant 2010 USD)


In [80]:
# Open files
# NY.GDP.PCAP.PP.KD is WDI indicator
'''
NY = national accounts: income
MKTP = market prices
PCAP = per capita
PP = purchasing power (no PP means not PP)
KD = constant (vs CD = current)
'''
# df_gdp is gdp market constant
gdp_mkt_const = pd.read_csv(zerg_path + 'data/wb/gdp/gdp_constant/API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv', skiprows=3)
gdp_meta = pd.read_csv(zerg_path + 'data/wb/gdp/gdp_constant/Metadata_Country_API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv')

In [81]:
# Get variables (column names)
gdp_mkt_const.columns # years 1960-2017
gdp_mkt_const.shape

(264, 63)

### Filter non countries (should return 217 entries) and subset only interested columns

In [82]:
gdp_country_only = filter_non_countries(gdp_mkt_const, gdp_meta)

df_gdp = select_relevant_cols(gdp_country_only)
df_gdp.shape # 217, 59

(217, 59)

### TASK: Get missing years/countries


### Ideas

1. Idea Red
    
    1) Get a list of countries that has any missing data, including # years missing
    
    2) Sort by years missing (country with largest missing data first)

2. Idea Orange (outcome data need)

    1) Check interesting countries
        Mexico
        Philipines
        USA
        Ethiopia
        Germany/West Germany
        Indonesia
        Soviet Union/Russia
        France
        Brazil
        Spain....
    
3. Idea Yellow (population as guage for important countries)
    1) Filter by population


In [91]:
del gdp_country_only['Unnamed: 5']
gdp_country_only.to_csv(zerg_path + 'data/combined_data/wb_gdp_mkt_constant_all_cols.csv')

In [92]:
df_gdp.to_csv(zerg_path + 'data/combined_data/wb_gdp_mkt_constant.csv')

In [83]:
df_gdp.iloc[1:5]
pd.isnull(df_gdp).any(1).nonzero() 

(array([  0,   1,   2,   3,   4,   5,   7,   8,   9,  12,  18,  19,  21,
         22,  24,  27,  28,  29,  33,  34,  42,  43,  45,  46,  47,  48,
         49,  50,  51,  52,  58,  60,  61,  65,  66,  69,  71,  72,  73,
         74,  75,  77,  78,  80,  82,  84,  85,  86,  88,  90,  91,  92,
         96,  97,  99, 101, 102, 103, 104, 106, 107, 108, 110, 111, 112,
        113, 115, 117, 118, 119, 120, 121, 122, 124, 126, 127, 128, 129,
        131, 132, 133, 134, 136, 139, 140, 147, 148, 149, 154, 155, 156,
        157, 158, 161, 162, 163, 164, 165, 167, 171, 173, 174, 175, 176,
        177, 178, 179, 180, 181, 183, 184, 186, 187, 191, 192, 193, 194,
        196, 198, 199, 200, 201, 204, 206, 207, 208, 209, 210, 211, 212, 213]),)

### TASK: Get list of years missing for a country

Super naive EDA



In [84]:
# Count nan
print(sorted(df_gdp.isnull().sum(axis=1).unique().tolist()))

[0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 47, 49, 56, 57]


#### Number of missing years range from 0 to 57.. T.T

There are 17 unique values

In [85]:
get_missing_years(df_gdp, 'Aruba')

# Count number of null values per year
df_gdp.isnull().sum()

# Count number of null values for each country
df_gdp.apply(lambda row: row.isnull(),axis=1)

df_gdp.iloc[0].isnull().sum()

56

### Approach 1: Create a filter for countries with > 10 mil pop (Mike's Suggestion)

In [86]:
df_pop.shape

(217, 59)

In [87]:
df_pop[df_pop > 10**6].shape

(217, 59)

In [88]:
df['1960'].value_counts()
type(df.iloc[0]['1960']) # numpy.float64

# single value check
np.isnan(df.iloc[0]['1960']) # True 


False